# Setup

In [ ]:
import pandas as pd
import itertools
import os 

CutOffDate = pd.to_datetime('2021-08-01', utc = 'UTC')
Recent1YearDate = pd.to_datetime('2020-08-01', utc = 'UTC')
Recent2YearDate = pd.to_datetime('2019-08-01', utc = 'UTC')
Recent3YearDate = pd.to_datetime('2018-08-01', utc = 'UTC')

cutoffDates = {
    1:Recent1YearDate, 
    2:Recent2YearDate, 
    3:Recent3YearDate,
}

# Settings

In [ ]:
MIN_TEXT_LEN = 0

# Get Website Data

In [ ]:
# def get_website2rvw(Folder):

#     suffix = ''
#     website2path = {
#         'healthgrades': 'healthgrades{}.p'.format(suffix), 
#         'vitals': 'vitals{}.p'.format(suffix), 
#         'ratemds': 'ratemds{}.p'.format(suffix), 
#         'yelp': 'yelp{}.p'.format(suffix),
#         # 'zocdoc': 'zocdoc{}.p'.format(suffix)
#     }
    
#     website2data = {}
#     for name, path in website2path.items():
#         path = os.path.join(Folder, path)
#         df = pd.read_pickle(path)
#         website2data[name] = df
#     return website2data
    
# Folder = 'Output/MinText{}'.format(MIN_TEXT_LEN)
# website2data = get_website2rvw(Folder)

# NPPES Profile Score

In [ ]:
import pandas as pd
import numpy as np
import missingno as msno
import itertools

# NPPES = pd.read_pickle('Data/NPPES/NPI2InfoMDDO.p')
# nppes_cols = ['NPI', 'FirstName', 'LastName']
# NPPES = NPPES[nppes_cols]
# print(NPPES.shape)
# NPPES

In [ ]:


# def get_list_sum(x, finalreview_cols):
#     tmp = x[finalreview_cols].to_list()
#     tmp_list = [i for i in tmp if type(i) == list]
#     if len(tmp_list) == 0:
#         return np.nan
#     else:
#         return list(itertools.chain(*tmp_list))
    
# def get_prfscore_avg(x, finalpflscore_cols):
#     tmp = x[finalpflscore_cols].to_list()
#     tmp = [i for i in tmp if not pd.isnull(i)]
#     # tmp_list = [i for i in tmp if type(i) == list]
#     if len(tmp) == 0:
#         return np.nan
#     else:
#         return np.mean(tmp)
    
# def get_NPPESReview(NPPES, website2data):
#     for website, df in website2data.items():
#         tmp = df[['NPI', 'final_review', 'reported_profile_score']]
#         tmp.columns = ['NPI', 'review_{}'.format(website), 'pflscore_{}'.format(website)]
#         NPPES = pd.merge(NPPES, tmp, on = 'NPI', how = 'left')
        
#     # all 
#     cols = ['review_{}'.format(i) for i in website2data if 'zocdoc' not in i]
#     NPPES['review_all'] = NPPES.apply(lambda x: get_list_sum(x, cols), axis = 1)
#     cols = ['pflscore_{}'.format(i) for i in website2data if 'zocdoc' not in i]
#     NPPES['pflscore_all'] = NPPES.apply(lambda x: get_prfscore_avg(x, cols), axis = 1)
    
#     # all(zocdoc)
#     # cols = ['review_{}'.format(i) for i in website2data]
#     # NPPES['review_all(zocdoc)'] = NPPES.apply(lambda x: get_list_sum(x, cols), axis = 1)
#     # cols = ['pflscore_{}'.format(i) for i in website2data]
#     # NPPES['pflscore_all(zocdoc)'] = NPPES.apply(lambda x: get_prfscore_avg(x, cols), axis = 1)
#     return NPPES


# NPPESReview = get_NPPESReview(NPPES, website2data)


In [ ]:
# NPPESReview

In [ ]:
# NPPESReview.to_pickle('Output/NPPESReview.p')

In [ ]:
NPPESReview = pd.read_pickle('Output/NPPESReview.p')

# Get Review Table

In [ ]:
# tmp3['ReviewPrior3Y'].values[0]

CutoffDate = Recent3YearDate
# CutoffDate = Recent1YearDate
# CutoffDate = Recent2YearDate


In [ ]:
review_col = 'review_all' # 'review_healthgrades' # 'review_vitals' #    #  'review_all'
pflscore_col = 'pflscore_all' # 'pflscore_healthgrades' # 'pflscore_vitals' #   #  'pflscore_vitals' #  

tmp = NPPESReview[['NPI', 'FirstName', 'LastName', review_col, pflscore_col]]
tmp2 = tmp[-tmp[pflscore_col].isna()]
tmp3 = tmp2[tmp2[review_col].apply(lambda x:len(x) > 0)]
tmp3 = tmp3.reset_index(drop = True)
tmp3['numReviews'] = tmp3[review_col].apply(lambda x: len(x))
tmp3 = tmp3.sort_values('numReviews', ascending = False).reset_index(drop = True)

tmp3['ReviewPrior3Y'] = tmp3[review_col].apply(lambda x: [i['ReviewScore'] for i in x if i['ReviewDate'] < CutoffDate])
tmp3['ReviewRecnt3Y'] = tmp3[review_col].apply(lambda x: [i['ReviewScore'] for i in x if i['ReviewDate'] >= CutoffDate])

tmp3['ReviewPrior3YPlfm'] = tmp3[review_col].apply(lambda x: [i['ReviewSource'] for i in x if i['ReviewDate'] < CutoffDate])
tmp3['ReviewRecnt3YPlfm'] = tmp3[review_col].apply(lambda x: [i['ReviewSource'] for i in x if i['ReviewDate'] >= CutoffDate])


tmp3['numRewPrior3Y'] = tmp3['ReviewPrior3Y'].apply(lambda x: len(x))
tmp3['numRewRecnt3Y'] = tmp3['ReviewRecnt3Y'].apply(lambda x: len(x))

tmp3


In [ ]:
# from scipy import stats as st

# def get_t_test_between_past_recent(past, rcnt):
    
#     '''
#     Defines the alternative hypothesis. The following options are available (default is ‘two-sided’):
#     ‘two-sided’: the means of the distributions underlying the samples are unequal.
#     ‘less’: the mean of the distribution underlying the first sample is less than the mean of the distribution underlying the second sample.
#     ‘greater’: the mean of the distribution underlying the first sample is greater than the mean of the distribution underlying the second sample.
#     '''
    
#     if type(past) != list or type(rcnt) != list: return np.nan
    
#     size_past = len(past)
#     size_rcnt = len(rcnt)
#     equal_var = True
    
#     if size_past >= 2 and size_rcnt >= 2:
#         t, p = st.ttest_ind(past, rcnt, equal_var = equal_var)
#         # if p > 0.05: # means are significantly different. 
#         if p <= 0.1: # reject H0: u1 = u2. 
#             return True
#         elif p > 0.1:
#             return False
#         else:
#             # print(p)
#             return np.nan
#             # return False
        
#     else:
#         return np.nan
    

In [ ]:
# tmp3['sg_diff'] = tmp3.apply(lambda x: get_t_test_between_past_recent(x['ReviewPrior3Y'], 
#                                                                       x['ReviewRecnt3Y']),
#                              axis = 1)

# tmp3

# Records

In [ ]:
# title = 'Physician_star_50Rew'
# title = 'Physician_star_20Rew'
# title = 'Physician_star_10Rew'
title = 'Physician_star'
# title = 'Physician'

print(title)
print(len(tmp3))


if title == 'Physician':
    df = tmp3
    
elif title == 'Physician_star':
    df = tmp3
    df = df[df['numRewPrior3Y'] >= 1]
    df = df[df['numRewRecnt3Y'] >= 1]
    
elif title == 'Physician_star_10Rew':
    df = tmp3
    df = df[df['numRewPrior3Y'] >= 1]
    df = df[df['numRewRecnt3Y'] >= 1]
    df = df[df['numReviews'] > 10]
    
elif title == 'Physician_star_20Rew':
    df = tmp3
    df = df[df['numRewPrior3Y'] >= 1]
    df = df[df['numRewRecnt3Y'] >= 1]
    df = df[df['numReviews'] > 20]
    
    
elif title == 'Physician_star_50Rew':
    df = tmp3
    df = df[df['numRewPrior3Y'] >= 1]
    df = df[df['numRewRecnt3Y'] >= 1]
    df = df[df['numReviews'] > 50]
    
print(len(df))

In [ ]:
df

In [ ]:
# df['review_all'].values[0]

In [ ]:
list2d = df.apply(lambda x: [x['NPI']] * x['numRewPrior3Y'], axis = 1).values
NPI = list(itertools.chain.from_iterable(list2d))

list2d = df['ReviewPrior3Y'].values
ReviewScore = list(itertools.chain.from_iterable(list2d))
assert len(NPI) == len(ReviewScore)

list2d = df['ReviewPrior3YPlfm'].values
ReviewPlatform = list(itertools.chain.from_iterable(list2d))
assert len(NPI) == len(ReviewPlatform)

df1 = pd.DataFrame({'NPI': NPI, 'ReviewScore': ReviewScore, 'plat': ReviewPlatform})
df1['rcnt'] = 0
df1['past'] = 1

list2d = df.apply(lambda x: [x['NPI']] * x['numRewRecnt3Y'], axis = 1).values
NPI = list(itertools.chain.from_iterable(list2d))

list2d = df['ReviewRecnt3Y'].values
ReviewScore = list(itertools.chain.from_iterable(list2d))
assert len(NPI) == len(ReviewScore)



list2d = df['ReviewRecnt3YPlfm'].values
ReviewPlatform = list(itertools.chain.from_iterable(list2d))
assert len(NPI) == len(ReviewPlatform)



df2 = pd.DataFrame({'NPI': NPI, 'ReviewScore': ReviewScore, 'plat': ReviewPlatform})
df2['rcnt'] = 1
df2['past'] = 0

all_date = pd.concat([df1, df2]).sort_values(['NPI']).reset_index(drop = True)
all_date


In [ ]:
plat = pd.get_dummies(all_date['plat'])
plat

In [ ]:
final = pd.concat([all_date, plat], axis = 1)
final

In [ ]:
path = 'LMER/data_{}_{}_new.csv'.format(review_col, title)
final.to_csv(path, index = False)
print(path)

# Focus on Physicians Whose Review Number Larger than X

In [ ]:
MIN_PHYSICIAN_REVIEW_NUM = 100 # 0 # 10, 20, 50

In [ ]:
review_num = NPPESReview['review_all'].apply(lambda x: len(x) if type(x) == list else np.nan)
review_num

In [ ]:
NPPESReviewX = NPPESReview[review_num >= MIN_PHYSICIAN_REVIEW_NUM].reset_index(drop = True)
NPPESReviewX

In [ ]:
print('{} Physicians have more than {} reviews'.format(len(NPPESReviewX), MIN_PHYSICIAN_REVIEW_NUM))

# Recent vs Past

In [ ]:
def get_recent_past_reviews(x, CutOffDate):
    recent = []
    past = []
    for i in x:
        if i['ReviewDate'] >= CutOffDate:
            recent.append(i)
        else:
            past.append(i)
    return recent, past

In [ ]:
def get_recent_past_review_score(col, NPPESReview, RecentYear, PAST_MIN_NUM):
    info = {}
    info['Platform'] = col.split('_')[1]
    
    HasRecentReview = 'HasReviewsRct{}Y'.format(RecentYear)
    CutOffDate = cutoffDates[RecentYear]
    # info['PAST_MIN_NUM'] = PAST_MIN_NUM
    info['Recent'] = 'Rct{}Y ({})'.format(RecentYear, str(CutOffDate.date()))
    
    
    # get profile info
    df = NPPESReview[['NPI', col]]
    df = df[-df[col].isna()].reset_index(drop = True)
    df['LatestReviewDate'] = df[col].apply(lambda x: np.max([i['ReviewDate'] for i in x]) if len(x) > 0 else np.nan)
    info['WebPhyNum'] = len(df)
    
    # has review
    df = df[-df['LatestReviewDate'].isna()].reset_index(drop = True)
    info['HasRvwNum'] = len(df)
    
    # has recent review
    df[HasRecentReview] = df['LatestReviewDate'].apply(lambda x: True if x >= CutOffDate else False)
    df = df[df[HasRecentReview]].reset_index(drop = True)
    info['HasRctNum'] = len(df)
    
    
    # has recent and past review and Filter by PAST_MIN_NUM
    recent_past = df[col].apply(lambda x: get_recent_past_reviews(x, CutOffDate))
    df['recent'] = [i[0] for i in recent_past.values]
    df['past'] = [i[1] for i in recent_past.values]
    df['recent_scores'] = df['recent'].apply(lambda x: [float(i['ReviewScore']) for i in x])
    df['past_scores'] = df['past'].apply(lambda x: [float(i['ReviewScore']) for i in x])
    
    df['recent_num'] = df['recent'].apply(lambda x: len(x))
    df['past_num'] = df['past'].apply(lambda x: len(x))
    
    df = df[df['past_num'] >= PAST_MIN_NUM].reset_index(drop = True)
    info['HasRctPst{}Num'.format(PAST_MIN_NUM)] = len(df)
    
    # Recent Score and Past Score
    df['recent_mean_score'] = df['recent_scores'].apply(lambda x: np.mean(x))
    df['past_mean_score'] = df['past_scores'].apply(lambda x: np.mean(x))
    
    # T, I, D, M
    df['rencetpast_diff'] = df['recent_mean_score'] - df['past_mean_score']
    increase_df = df[df['rencetpast_diff'] >= 0.5].reset_index(drop = True)
    decrease_df = df[df['rencetpast_diff'] <= -0.5].reset_index(drop = True)
    unchange_df = df[df['rencetpast_diff'].apply(lambda x: x > -0.5 and x < 0.5)].reset_index(drop = True)
    
    info['T'] = '{} ({:.2f}%)'.format(len(df), 100*len(df)/len(df))
    info['T_recentscore'] = '{:.2f} ({:.2f})'.format(df['recent_mean_score'].mean(), df['recent_mean_score'].std())
    info['T_pastscore'] = '{:.2f} ({:.2f})'.format(df['past_mean_score'].mean(), df['past_mean_score'].std())
    rct = df['recent_num'].sum(); all = df['recent_num'].sum() + df['past_num'].sum()
    info['T_recentrvw/allrvw'] = '{}/{} ({:.2f}%)'.format(rct, all, 100 * rct/all)
    
    
    
    info['D'] = '{} ({:.2f}%)'.format(len(decrease_df), 100*len(decrease_df)/len(df))
    info['D_recentscore'] = '{:.2f} ({:.2f})'.format(decrease_df['recent_mean_score'].mean(), decrease_df['recent_mean_score'].std())
    info['D_pastscore'] = '{:.2f} ({:.2f})'.format(decrease_df['past_mean_score'].mean(), decrease_df['past_mean_score'].std())
    rct = decrease_df['recent_num'].sum(); all = decrease_df['recent_num'].sum() + decrease_df['past_num'].sum()
    info['D_recentrvw/allrvw'] = '{}/{} ({:.2f}%)'.format(rct, all, 100 * rct/all)
    
    
    info['M'] = '{} ({:.2f}%)'.format(len(unchange_df), 100*len(unchange_df)/len(df))
    info['M_recentscore'] = '{:.2f} ({:.2f})'.format(unchange_df['recent_mean_score'].mean(), unchange_df['recent_mean_score'].std())
    info['M_pastscore'] = '{:.2f} ({:.2f})'.format(unchange_df['past_mean_score'].mean(), unchange_df['past_mean_score'].std())
    rct = unchange_df['recent_num'].sum(); all = unchange_df['recent_num'].sum() + unchange_df['past_num'].sum()
    info['M_recentrvw/allrvw'] = '{}/{} ({:.2f}%)'.format(rct, all, 100 * rct/all)
    
    
    
    info['I'] = '{} ({:.2f}%)'.format(len(increase_df), 100*len(increase_df)/len(df))
    info['I_recentscore'] = '{:.2f} ({:.2f})'.format(increase_df['recent_mean_score'].mean(), increase_df['recent_mean_score'].std())
    info['I_pastscore'] = '{:.2f} ({:.2f})'.format(increase_df['past_mean_score'].mean(), increase_df['past_mean_score'].std())
    rct = increase_df['recent_num'].sum(); all = increase_df['recent_num'].sum() + increase_df['past_num'].sum()
    info['I_recentrvw/allrvw'] = '{}/{} ({:.2f}%)'.format(rct, all, 100* rct/all)
    

    return info


In [ ]:
ReviewsInfo = NPPESReviewX
phynum = 'Review{}'.format(MIN_PHYSICIAN_REVIEW_NUM)
folder = 'Output/RecentVSOld'

for PAST_MIN_NUM in [5, 1]:
    for RecentYear in [1, 3]:
        L = []
        for col in [i for i in ReviewsInfo.columns if 'review' in i]:
            # print(col, RecentYear)
            info = get_recent_past_review_score(col, ReviewsInfo, RecentYear, PAST_MIN_NUM)
            L.append(info)

        Recent_Past_Report = pd.DataFrame(L)
        path = '{}/{}-MinTxt{}_RctY{}_PastRvw{}_Report.csv'.format(folder, phynum, MIN_TEXT_LEN, RecentYear, PAST_MIN_NUM)
        print('save to: {}'.format(path))
        Recent_Past_Report.to_csv(path, index = False)

Recent_Past_Report

# Recent vs Profile

In [ ]:

def get_recent_profile_review_score(col, NPPESReview, RecentYear, PAST_MIN_NUM):
    info = {}
    info['Platform'] = col.split('_')[1]
    
    HasRecentReview = 'HasReviewsRct{}Y'.format(RecentYear)
    CutOffDate = cutoffDates[RecentYear]
    # info['PAST_MIN_NUM'] = PAST_MIN_NUM
    info['Recent'] = 'Rct{}Y ({})'.format(RecentYear, str(CutOffDate.date()))
    
    
    # get profile info
    pflscore_col = col.replace('review', 'pflscore')
    df = NPPESReview[['NPI', col, pflscore_col]]
    df = df[-df[col].isna()].reset_index(drop = True)
    df['LatestReviewDate'] = df[col].apply(lambda x: np.max([i['ReviewDate'] for i in x]) if len(x) > 0 else np.nan)
    info['WebPhyNum'] = len(df)
    
    # has review
    df = df[-df['LatestReviewDate'].isna()].reset_index(drop = True)
    info['HasRvwNum'] = len(df)
    
    # has recent review
    df[HasRecentReview] = df['LatestReviewDate'].apply(lambda x: True if x >= CutOffDate else False)
    df = df[df[HasRecentReview]].reset_index(drop = True)
    info['HasRctNum'] = len(df)
    
    
    # has recent and past review and Filter by PAST_MIN_NUM
    recent_past = df[col].apply(lambda x: get_recent_past_reviews(x, CutOffDate))
    df['recent'] = [i[0] for i in recent_past.values]
    df['past'] = [i[1] for i in recent_past.values]
    df['recent_scores'] = df['recent'].apply(lambda x: [float(i['ReviewScore']) for i in x])
    
    df['recent_num'] = df['recent'].apply(lambda x: len(x))
    df['past_num'] = df['past'].apply(lambda x: len(x))
    df = df[df['past_num'] >= PAST_MIN_NUM].reset_index(drop = True)
    info['HasRctPst{}Num'.format(PAST_MIN_NUM)] = len(df)
    
    # Recent Score and Past Score
    df['recent_mean_score'] = df['recent_scores'].apply(lambda x: np.mean(x))
    df['profile_score'] = df[pflscore_col].apply(lambda x: float(x))
    
    # T, I, D, M
    df['rencetprfl_diff'] = df['recent_mean_score'] - df['profile_score']
    increase_df = df[df['rencetprfl_diff'] >= 0.5].reset_index(drop = True)
    decrease_df = df[df['rencetprfl_diff'] <= -0.5].reset_index(drop = True)
    unchange_df = df[df['rencetprfl_diff'].apply(lambda x: x > -0.5 and x < 0.5)].reset_index(drop = True)
    
    info['T'] = '{} ({:.2f}%)'.format(len(df), 100*len(df)/len(df))
    info['T_recentscore'] = '{:.2f} ({:.2f})'.format(df['recent_mean_score'].mean(), df['recent_mean_score'].std())
    info['T_profilescore'] = '{:.2f} ({:.2f})'.format(df['profile_score'].mean(), df['profile_score'].std())
    rct = df['recent_num'].sum(); all = df['recent_num'].sum() + df['past_num'].sum()
    info['T_recentrvw/allrvw'] = '{}/{} ({:.2f}%)'.format(rct, all, 100 * rct/all)
    
    
    info['D'] = '{} ({:.2f}%)'.format(len(decrease_df), 100*len(decrease_df)/len(df))
    info['D_recentscore'] = '{:.2f} ({:.2f})'.format(decrease_df['recent_mean_score'].mean(), decrease_df['recent_mean_score'].std())
    info['D_profilescore'] = '{:.2f} ({:.2f})'.format(decrease_df['profile_score'].mean(), decrease_df['profile_score'].std())
    rct = decrease_df['recent_num'].sum(); all = decrease_df['recent_num'].sum() + decrease_df['past_num'].sum()
    info['D_recentrvw/allrvw'] = '{}/{} ({:.2f}%)'.format(rct, all, 100 * rct/all)
    
    
    
    info['M'] = '{} ({:.2f}%)'.format(len(unchange_df), 100*len(unchange_df)/len(df))
    info['M_recentscore'] = '{:.2f} ({:.2f})'.format(unchange_df['recent_mean_score'].mean(), unchange_df['recent_mean_score'].std())
    info['M_profilescore'] = '{:.2f} ({:.2f})'.format(unchange_df['profile_score'].mean(), unchange_df['profile_score'].std())
    rct = unchange_df['recent_num'].sum(); all = unchange_df['recent_num'].sum() + unchange_df['past_num'].sum()
    info['M_recentrvw/allrvw'] = '{}/{} ({:.2f}%)'.format(rct, all, 100 * rct/all)
    
    
    
    info['I'] = '{} ({:.2f}%)'.format(len(increase_df), 100*len(increase_df)/len(df))
    info['I_recentscore'] = '{:.2f} ({:.2f})'.format(increase_df['recent_mean_score'].mean(), increase_df['recent_mean_score'].std())
    info['I_profilescore'] = '{:.2f} ({:.2f})'.format(increase_df['profile_score'].mean(), increase_df['profile_score'].std())
    rct = increase_df['recent_num'].sum(); all = increase_df['recent_num'].sum() + increase_df['past_num'].sum()
    info['I_recentrvw/allrvw'] = '{}/{} ({:.2f}%)'.format(rct, all, 100 * rct/all)
    
    return info


In [ ]:
ReviewsInfo = NPPESReviewX
phynum = 'Review{}'.format(MIN_PHYSICIAN_REVIEW_NUM)

folder = 'Output/RecentVSProfile'


for PAST_MIN_NUM in [5, 1]: # [5, 1]
    for RecentYear in [3]: #[1, 2, 3]:
        L = []
        for col in [i for i in ReviewsInfo.columns if 'review' in i]:
            # print(col, RecentYear)
            info = get_recent_profile_review_score(col, ReviewsInfo, RecentYear, PAST_MIN_NUM)
            L.append(info)

        Recent_Profile_Report = pd.DataFrame(L)
        path = '{}/{}-MinTxt{}_RctY{}_PastRvw{}_Report.csv'.format(folder, phynum, MIN_TEXT_LEN, RecentYear, PAST_MIN_NUM)
        print('save to: {}'.format(path))
        Recent_Profile_Report.to_csv(path, index = False)

Recent_Profile_Report

# Report

In [ ]:
# import pandas as pd
# path = 'Output/RecentVSProfile/Review0-MinTxt0_RctY3_PastRvw1_Report.csv'
# DF = pd.read_csv(path)
# DF

In [ ]:
import pandas as pd

def get_all_info(DF):
    
    for idx, row in DF.iterrows():
        d = row.to_dict()
        if d['Platform'] == 'all':
            return d
        
def get_Info(d):
    L = []
    for i in ['T', 'D', 'M', 'I']:
        info = {k:v for k, v in d.items() if i in k}
        # print(info)

        x = {k.replace(i+ '_', '').replace(i, 'GroupMember'): v for k, v in info.items()}
        x['Group'] = i
        L.append(x)
        
    Report = pd.DataFrame(L)
    Report = Report[['Group', 'GroupMember', 'recentscore', 'profilescore', 'recentrvw/allrvw']]
    return Report


T2Full = {
    'T': 'Physician Having Recent Reviews', 
    'D': 'Lower Recent Score than Profile Score',
    'M': 'Similar Recent Score to Profile Score',
    'I': 'Higher Recent Score to Profile Score'
}

def get_clean_result(d):
    Report = get_Info(d)

    L = []
    for idx, row in Report.iterrows():
        d = row.to_dict()
        if d['Group'] == 'T':
            d['Name'] = T2Full[d['Group']] + ' (' + d['GroupMember'].split(' ')[0] + ')'
        else:
            d['Name'] = T2Full[d['Group']] + ' ' + d['GroupMember'].split(' ')[1] 
        d['Group'] = T2Full[d['Group']]
        d['Group Member'] = int(d['GroupMember'].split(' ')[0])
        d['Recent Score'] = float(d['recentscore'].split(' ')[0])
        d['Profile Score'] = float(d['profilescore'].split(' ')[0])
        rctrvw, rvw = d['recentrvw/allrvw'].split(' ')[0].split('/')
        d['RctRvw/Physican'] = round(int(rctrvw) / d['Group Member'], 2)
        d['Rvw/Physican'] = round(int(rvw) / d['Group Member'], 2)
        d.pop('Group Member')
        L.append(d)

    Report = pd.DataFrame(L)
    
    return Report



In [ ]:

path = 'Output/RecentVSProfile/Review0-MinTxt0_RctY3_PastRvw1_Report.csv'
DF = pd.read_csv(path)

d = get_all_info(DF)
Report = get_clean_result(d)
Report.to_clipboard()

Report

In [ ]:

path = 'Output/RecentVSProfile/Review10-MinTxt0_RctY3_PastRvw1_Report.csv'
DF = pd.read_csv(path)

d = get_all_info(DF)
Report = get_clean_result(d)
Report.to_clipboard()

Report

In [ ]:

path = 'Output/RecentVSProfile/Review20-MinTxt0_RctY3_PastRvw1_Report.csv'
DF = pd.read_csv(path)

d = get_all_info(DF)
Report = get_clean_result(d)
Report.to_clipboard()

Report

In [ ]:

path = 'Output/RecentVSProfile/Review50-MinTxt0_RctY3_PastRvw5_Report.csv'
DF = pd.read_csv(path)

d = get_all_info(DF)
Report = get_clean_result(d)
Report.to_clipboard()

Report

In [ ]:

path = 'Output/RecentVSProfile/Review100-MinTxt0_RctY3_PastRvw1_Report.csv'
DF = pd.read_csv(path)

d = get_all_info(DF)
Report = get_clean_result(d)
Report.to_clipboard()

Report

# Parking Lot

In [ ]:
# df = NPPESReview[['NPI', 'FirstName', 'LastName', 'review_all', 'pflscore_all']].reset_index(drop = True)
# df['review_all_num'] = df['review_all'].apply(lambda x: len(x) if type(x) == list else 0)

# df = df[['NPI', 'FirstName', 'LastName', 'review_all_num', 'pflscore_all']]

# df.to_csv('npi2rating_text.csv', index = False, sep = '\t')
# df

In [ ]:
# df = df.rename(columns = {'review_all_num': 'review_text_num'})
# df

In [ ]:
# df2 = pd.read_csv('npi2review_num.csv', sep = '\t')
# df2

In [ ]:
# df_new = pd.merge(df2, df[['NPI', 'review_text_num']])

# cols = ['NPI', 'FirstName', 'LastName','pflscore_all', 'review_all_num', 'review_text_num']

# df_new = df_new[cols]
# df_new.to_csv('npi2review_info.csv', index = False, sep = '\t')
# df_new

In [ ]:
# from scipy import stats as st

# '''
# Defines the alternative hypothesis. The following options are available (default is ‘two-sided’):

# ‘two-sided’: the means of the distributions underlying the samples are unequal.

# ‘less’: the mean of the distribution underlying the first sample is less than the mean of the distribution underlying the second sample.

# ‘greater’: the mean of the distribution underlying the first sample is greater than the mean of the distribution underlying the second sample.

# '''
# def get_t_test_between_past_recent(x, cut_off_date):
#     if type(x) != list:
#         return None
    
#     d = {}
    
#     past = [i['ReviewScore'] for i in x if i['ReviewDate'] < cut_off_date]
#     rcnt = [i['ReviewScore'] for i in x if i['ReviewDate'] >= cut_off_date]

#     size_past = len(past)
#     size_rcnt = len(rcnt)
    
    
#     equal_var = False
#     if size_past >= 1 and size_rcnt >= 1:
        
#         d['mean_past'] = np.mean(past)
#         d['mean_rcnt'] = np.mean(rcnt)
#         d['size_past'] = size_past
#         d['size_rcnt'] = size_rcnt
        
#         # Null Hypo: u1 = u2, Alternative Hypo: u1 != u2
#         # We want to select the samples that cannot support Null Hypo
#         # That is p >= 0.05.
#         t, p = st.ttest_ind(past, rcnt, equal_var = equal_var)
#         d['p-2sides'] = p
#         d['p-2sides**'] = p <= 0.05
        
#         # t, p = st.ttest_ind(past, [i-0.5 for i in rcnt], equal_var = equal_var, alternative = 'less')
#         # d['p-1side_P<R-0.5'] = p
#         # d['p-1side_P<R-0.5**'] = p <= 0.05
        
#         # t, p = st.ttest_ind(past, [i+0.5 for i in rcnt], equal_var = equal_var, alternative = 'greater')
#         # d['p-1side_P>R+0.5'] = p
#         # d['p-1side_P>R+0.5**'] = p <= 0.05

            
        
# #         if d['mean_rcnt'] - d['mean_past'] >= 0.5: # 
# #             d['Changed'] = 'Increased'
# #             # H0: whether past >= recent - 0.5. (p <=0.05 rejects it.)
# #             # H1: past < recent - 0.5 (less)
# #             # We want select the samples that support H0, i.e. p <= 0.05
# #             t, p = st.ttest_ind(past, [i-0.5 for i in rcnt], equal_var = equal_var, alternative = 'less')
# #             d['p-1side'] = p
# #             d['p-1side**'] = p <= 0.05
            
# #         elif d['mean_past'] - d['mean_rcnt'] >= 0.5:
# #             d['Changed'] = 'Decreased'
# #             # H0: whether past <= recent + 0.5 (small p rejects it)
# #             # H1: past > recent + 0.5. (greater)
# #             t, p = st.ttest_ind(past, [i+0.5 for i in rcnt], equal_var = equal_var, alternative = 'greater')
# #             d['p-1side'] = p
# #             d['p-1side**'] = p <= 0.05
            
# #         else:
# #             d['Changed'] = 'Unchanged'
            
        
#         return d
    
#     else:
#         return None
    
    

In [ ]:
st.ttest_ind?

In [ ]:
# x = NPPESReview['review_all'].values[1]
# x


# cut_off_date = cutoffDates[1]
# print(cut_off_date)
# get_t_test_between_past_recent(x, cut_off_date)


# cut_off_date = cutoffDates[1]
# ttest_report = NPPESReview['review_all'].apply(lambda x: get_t_test_between_past_recent(x, cut_off_date))

# report = pd.DataFrame(ttest_report.dropna().to_list())
# print(len(report))
# report = report[-report['p-2sides'].isna()].reset_index(drop = True)
# print(len(report))
# print(report['p-2sides**'].value_counts().to_dict())
# # print(report['p-1side_P<R-0.5**'].value_counts().to_dict())
# # print(report['p-1side_P>R+0.5**'].value_counts().to_dict())
# report


In [ ]:
# cut_off_date = cutoffDates[2]
# ttest_report = NPPESReview['review_all'].apply(lambda x: get_t_test_between_past_recent(x, cut_off_date))

# report = pd.DataFrame(ttest_report.dropna().to_list())
# print(len(report))
# report = report[-report['p-2sides'].isna()].reset_index(drop = True)
# print(len(report))
# print(report['p-2sides**'].value_counts().to_dict())
# # print(report['p-1side_P<R-0.5**'].value_counts().to_dict())
# # print(report['p-1side_P>R+0.5**'].value_counts().to_dict())
# report


In [ ]:
# cut_off_date = cutoffDates[3]
# ttest_report = NPPESReview['review_all'].apply(lambda x: get_t_test_between_past_recent(x, cut_off_date))

# report = pd.DataFrame(ttest_report.dropna().to_list())
# print(len(report))
# report = report[-report['p-2sides'].isna()].reset_index(drop = True)
# print(len(report))
# print(report['p-2sides**'].value_counts().to_dict())
# # print(report['p-1side_P<R-0.5**'].value_counts().to_dict())
# # print(report['p-1side_P>R+0.5**'].value_counts().to_dict())
# report

